In [1]:
import pandas as pd

In [2]:
from sentence_transformers import SentenceTransformer

/Users/amoghkulkarni/.local/share/virtualenvs/python_practice-ZCCdr_2d/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
model_name = 'multi-qa-mpnet-base-dot-v1'

In [4]:
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
gpt_4o_mini_df = pd.read_csv('results-gpt4o-mini.csv')

In [7]:
answer_llm = gpt_4o_mini_df.iloc[0].answer_llm

In [8]:
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [9]:
answer_embedding = embedding_model.encode(answer_llm)

In [10]:
len(answer_embedding)

768

In [11]:
answer_embedding[0]

-0.42244664

In [22]:
evaluations = []

In [18]:
gpt_4o_mini_df = gpt_4o_mini_df.iloc[:300]

In [19]:
gpt_40_results = gpt_4o_mini_df.to_dict(orient='records')

In [15]:
gpt_40_results[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [16]:
from tqdm.auto import tqdm

In [23]:
for each_item in tqdm(gpt_40_results):
    llm_embed = embedding_model.encode(each_item['answer_llm'])
    orig_embed = embedding_model.encode(each_item['answer_orig'])

    evaluations.append(llm_embed.dot(orig_embed))

  0%|          | 0/300 [00:00<?, ?it/s]

In [24]:
len(evaluations)

300

In [25]:
gpt_4o_mini_df['evaluation'] = evaluations

/var/folders/zw/mqwlds_s7ml5_fvqptkb174h0000gn/T/ipykernel_20355/1776927725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_4o_mini_df['evaluation'] = evaluations


In [27]:
gpt_4o_mini_df['evaluation'].quantile(0.75)

31.674302101135254

In [28]:
import numpy as np

In [33]:
norm = np.sqrt((evaluations.dot(evaluations).sum())

AttributeError: 'list' object has no attribute 'dot'

In [37]:
v1 = embedding_model.encode(gpt_40_results[0]['answer_llm'])

In [38]:
np.sqrt((v1 * v1).sum())

5.900879

In [40]:
evaluations_array = np.array(evaluations)

In [43]:
norm = np.sqrt((evaluations_array * evaluations_array).sum())

In [50]:
type(evaluations)

list

In [46]:
gpt_4o_mini_df['evaluation_new'] = gpt_4o_mini_df['evaluation'] / norm

/var/folders/zw/mqwlds_s7ml5_fvqptkb174h0000gn/T/ipykernel_20355/2337137279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_4o_mini_df['evaluation_new'] = gpt_4o_mini_df['evaluation'] / norm


In [48]:
gpt_4o_mini_df['evaluation_new'].quantile(0.75)

0.06479034014046192

In [52]:
evaluations_normalized = []

In [53]:
for each_item in tqdm(gpt_40_results):
    llm_embed = embedding_model.encode(each_item['answer_llm'])
    llm_embed_array = np.array(llm_embed)
    llm_embed_norm = np.sqrt((llm_embed_array * llm_embed_array).sum())
    llm_embed = llm_embed / llm_embed_norm

    orig_embed = embedding_model.encode(each_item['answer_orig'])
    orig_embed_array = np.array(orig_embed)
    orig_embed_norm = np.sqrt((orig_embed_array * orig_embed_array).sum())
    orig_embed = orig_embed / orig_embed_norm

    evaluations_normalized.append(llm_embed.dot(orig_embed))

  0%|          | 0/300 [00:00<?, ?it/s]

In [54]:
evaluations_normalized[:5]

[0.5067543, 0.38854897, 0.7185991, 0.33726633, 0.5217922]

In [55]:
gpt_4o_mini_df['evaluation_norm'] = evaluations_normalized

/var/folders/zw/mqwlds_s7ml5_fvqptkb174h0000gn/T/ipykernel_20355/3749032999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpt_4o_mini_df['evaluation_norm'] = evaluations_normalized


In [57]:
gpt_4o_mini_df['evaluation_norm'].quantile(0.75)

0.8362347185611725

In [58]:
from rouge import Rouge

In [59]:
rouge_scorer = Rouge()

In [78]:
row = gpt_4o_mini_df.iloc[11]

In [79]:
scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]

In [80]:
scores

{'rouge-1': {'r': 0.6060606060606061,
  'p': 0.6060606060606061,
  'f': 0.6060606010606061},
 'rouge-2': {'r': 0.43243243243243246,
  'p': 0.41025641025641024,
  'f': 0.42105262658241},
 'rouge-l': {'r': 0.5757575757575758,
  'p': 0.5757575757575758,
  'f': 0.5757575707575758}}

In [66]:
f_scores = []
for each in scores:
    f_scores.append(scores[each]['f'])

In [69]:
sum(f_scores) / len(f_scores)

0.35490034990035496

In [81]:
rouge_scores_list = []

In [73]:
for rouge in scores:
    for each_value in scores[rouge]:
        rouge_key = f'{rouge}-{each_value}'
        rouge_value = scores[rouge][each_value]
        print(f'{rouge_key} - {rouge_value}')

rouge-1-r - 0.45454545454545453
rouge-1-p - 0.45454545454545453
rouge-1-f - 0.45454544954545456
rouge-2-r - 0.21621621621621623
rouge-2-p - 0.21621621621621623
rouge-2-f - 0.21621621121621637
rouge-l-r - 0.3939393939393939
rouge-l-p - 0.3939393939393939
rouge-l-f - 0.393939388939394


In [84]:
for each_record in tqdm(gpt_40_results):
    temp_rouge_score = {
        'document': each_record['document']
    }
    scores = rouge_scorer.get_scores(each_record['answer_llm'], each_record['answer_orig'])[0]
    for rouge in scores:
        for each_value in scores[rouge]:
            rouge_key = f'{rouge}-{each_value}'
            rouge_value = scores[rouge][each_value]
            temp_rouge_score.update({ rouge_key: rouge_value })
    rouge_scores_list.append(temp_rouge_score)

  0%|          | 0/300 [00:00<?, ?it/s]

In [86]:
rouge_values_df = pd.DataFrame(rouge_scores_list)

In [91]:
rouge_values_df.describe()

,rouge-1-r,rouge-1-p,rouge-1-f,rouge-2-r,rouge-2-p,rouge-2-f,rouge-l-r,rouge-l-p,rouge-l-f
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.357817,0.472618,0.378844,0.198613,0.258626,0.206965,0.334597,0.440623,0.353807
std,0.182520,0.194116,0.165977,0.164964,0.174559,0.153550,0.177885,0.189329,0.162965
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.220668,0.357143,0.261625,0.074632,0.138093,0.097809,0.199180,0.326420,0.228032
50%,0.341784,0.485281,0.378762,0.159075,0.230769,0.178671,0.317073,0.440000,0.337792
75%,0.470588,0.600000,0.479281,0.260995,0.335366,0.286181,0.440656,0.567045,0.451613
max,0.896552,1.000000,0.850000,0.805556,1.000000,0.739130,0.896552,1.000000,0.850000
